# Pull/Generate Camera Extrinsics & Intrinsics from Different Sources & Evaluate the Quality of the Extrinsics &Intrinsics Pairs using the BEV Image

In [1]:
%env WAYVE_PRODUCTION=true

env: WAYVE_PRODUCTION=true


In [2]:
from wayve.core.interfaces.protobuf.sensor_calibration_pb2 import CalibrationState, SensorStatus
from wayve.interfaces.protobuf.vehicle_calibration_pb2 import VehicleCalibration
from wayve.core.data.taxonomy import RunId
from wayve.core.calibration.load_calibration import load_calibration, load_calibration_textproto
from wayve.core.calibration.backcompat_support import convert_to_v1
from wayve.core.common.logger import get_logger
from wayve.services.data.lakehouse.pipeline_dag.tables import KEY_CALIBRATION_TABLE
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
from wayve.services.calibration.tools.verify_calibration import manual_verification
from wayve.core.data.reference_frames import ReferenceFrame
from pathlib import Path
import glob
import os

from typing import Any, Optional, Union


INFO     2024-07-30 17:41:01,580 wayve.core.calibration.utils get_calibration_database_url message=using prod database
INFO     2024-07-30 17:41:01,583 wayve.core.calibration.utils get_calibration_database_url message=using prod database
/home/benjin/.cache/bazel/_bazel_benjin/253920fd9b6cbb686fc69e16c3d2789e/execroot/WayveCode/bazel-out/k8-opt/bin/tools/jupyter_calibration.runfiles/pip-core_pydantic/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_trace" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
INFO     2024-07-30 17:41:06,203 azure.identity._credentials.environment No environment configuration found.


In [3]:
import importlib
importlib.reload(manual_verification)
ManualVerificationManager = manual_verification.ManualVerificationManager

In [4]:
_LOGGER = get_logger(__name__)

RUN_ID = "elbe/2024-07-25--10-07-27--gen2-d97313d5-521a-40c4-a3af-cd9ba31363b4"
OUTPUT_DIR = Path(f"/mnt/remote/data/benjin/extrinsics_intrinsics_eval/{RUN_ID}/")

COLMAP_MACHES = ["ebro", "elbe", "ganges", "jordan", "mekong", "mono", "napo", "orinoco", "thames"]
RECORDS_DIR = Path("/home/benjin/Development/wcs/WayveCode/wayve/core/calibration/records")

In [5]:
def generate_bev_overlay_for_run_id_and_calibration(run_id: str, output_directory: Path, calibration: Union[CalibrationState, VehicleCalibration]):
    if isinstance(calibration, VehicleCalibration):
        calibration = convert_to_v1(calibration)
    
    assert isinstance(calibration, CalibrationState)

    cvm = ManualVerificationManager(
        calibration=calibration,
        run_id=RunId.from_string(run_id),
        use_geo_tagging=True,
        ckpt_threshold_m=None,
    )

    cvm._cameras_to_check = set((
        ReferenceFrame.CameraFrontForward, ReferenceFrame.CameraLeftForward, ReferenceFrame.CameraRightForward))

    _ = cvm.manual_verify_bev_overlay(png_dir=output_directory)

def get_colmap_extrinsics_intrinsics(run_id: str) -> CalibrationState:
    """
    The latest records files for our mach-es contain colmap extrinsics and intrinsics
    apart from colorado.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)

    vehicle = run_id.run_vehicle_id

    if vehicle not in COLMAP_MACHES:
        raise ValueError(f"Vehicle {vehicle} did not have colmap extrinsics and intrinsics generated")
    
    # Gets the most recent records calibration for the vehicle that is before the run_datetime
    # Need to take care to not chose a run_id from a mach-e that is before the colmap calibration
    cal_state = load_calibration(vehicle, run_id.run_datetime)

    return cal_state

def get_initial_record_path(vehicle_records_dir: Path) -> Path:
    # Get a list of all files in the directory
    files = glob.glob(os.path.join(vehicle_records_dir.as_posix(), '*'))
    
    # Filter out directories, keeping only files
    files = [f for f in files if os.path.isfile(f)]
    
    # Sort files lexicographically
    files.sort()

    return files[0]

def half_the_resolution(calibration: CalibrationState) -> CalibrationState:
    new_calibration = CalibrationState()
    new_calibration.header.CopyFrom(calibration.header)

    for sensor_status in calibration.state:
        if len(sensor_status.sensors) > 1:
            new_calibration.state.append(sensor_status)
            continue

        if sensor_status.sensors[0] not in ["left-backward", "left-forward", "front-forward", "right-forward", "right-backward"]:
            new_calibration.state.append(sensor_status)
            continue

        # Create a new SensorStatus object and copy the original one
        new_sensor_status = SensorStatus()
        new_sensor_status.CopyFrom(sensor_status)
        
        # Modify the camera_matrix field
        new_sensor_status.camera_matrix[0] /= 2
        new_sensor_status.camera_matrix[2] /= 2
        new_sensor_status.camera_matrix[4] /= 2
        new_sensor_status.camera_matrix[5] /= 2
        
        # Add the modified SensorStatus to the new CalibrationState
        new_calibration.state.append(new_sensor_status)

    return new_calibration

def get_CAD_extrinsics_entron_intrinsics(run_id: str, records_dir: Path) -> CalibrationState:
    """
    Assumption is that the initial records files for our mach-es contain CAD extrinsics and entron intrinsics.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)
    
    vehicle = run_id.run_vehicle_id

    initial_record_path = get_initial_record_path(records_dir / vehicle)

    cad_extrinsics_entron_intrinsics = load_calibration_textproto(Path(initial_record_path))

    # The resolution of the cameras have halved since commissioning for all cameras except left-leftward and right-rightward
    return half_the_resolution(cad_extrinsics_entron_intrinsics)

def get_CAD_extrinsics_colmap_intrinsics(run_id: str, records_dir: Path) -> CalibrationState:
    """
    Assumption is that the initial records files for our mach-es contain CAD extrinsics and 
    and that the most recent record files contain colmap intrinsics.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)
    
    cad_extrinsics_entron_intrinsics = get_CAD_extrinsics_entron_intrinsics(run_id, records_dir)
    colmap_extrinsics_intrinsics = get_colmap_extrinsics_intrinsics(run_id)

    new_calibration = CalibrationState()
    new_calibration.header.CopyFrom(cad_extrinsics_entron_intrinsics.header)

    for sensor_status in cad_extrinsics_entron_intrinsics.state:
        if len(sensor_status.sensors) > 1:
            new_calibration.state.append(sensor_status)
            continue

        if sensor_status.sensors[0] not in ["left-backward", "left-forward", "front-forward", 
                                            "right-forward", "right-backward", "left-leftward", "right-rightward"]:
            new_calibration.state.append(sensor_status)
            continue

        # Create a new SensorStatus object and copy the original one
        new_sensor_status = SensorStatus()
        new_sensor_status.CopyFrom(sensor_status)
        
        # Replace the entron intrinsics with the colmap inrinsics
        # TBC


        new_sensor_status.camera_matrix[0] /= 2
        new_sensor_status.camera_matrix[2] /= 2
        new_sensor_status.camera_matrix[4] /= 2
        new_sensor_status.camera_matrix[5] /= 2
        
        # Add the modified SensorStatus to the new CalibrationState
        new_calibration.state.append(new_sensor_status)

    return new_calibration 

def get_two_cam_solve_calibration(run_id: str, intrinsics: Optional[Any] = None) -> Union[CalibrationState, VehicleCalibration]:
    pass

colmap_extrinsics_intrinsics = get_colmap_extrinsics_intrinsics(RUN_ID)
generate_bev_overlay_for_run_id_and_calibration(
    RUN_ID, 
    OUTPUT_DIR / "colmap_extrinsics_intrinsics", 
    colmap_extrinsics_intrinsics
)

# cad_extrinsics_entron_intrinsics = get_CAD_extrinsics_entron_intrinsics(RUN_ID, RECORDS_DIR) 
# generate_bev_overlay_for_run_id_and_calibration(
#     RUN_ID, 
#     OUTPUT_DIR / "cad_extrinsics_entron_intrinsics", 
#     cad_extrinsics_entron_intrinsics
# )

/home/benjin/.cache/bazel/_bazel_benjin/253920fd9b6cbb686fc69e16c3d2789e/execroot/WayveCode/bazel-out/k8-opt/bin/tools/jupyter_calibration.runfiles/WayveCode/wayve/services/calibration/tools/verify_calibration/verification_manager.py:137: UserWarning: CalibrationVerificationManager has not been passed template_calibration it will therefore not perform reasonable extrinsic checking
  warnings.warn(
INFO     2024-07-30 17:41:19,428 wayve.services.calibration.tools.verify_calibration.utils get_checkpoints_single_run message=Latitude 51.5514569 and longitude -0.1259598 of selected checkpoint
WARNING  2024-07-30 17:41:22,252 wayve.services.calibration.tools.verify_calibration.run_image_controller get_images message=Given run passes by 1 of the given checkpoints (requested: 3). Filling up the remaining images with strided images...
INFO     2024-07-30 17:41:24,182 wayve.services.data.common.azure.credentials get_azure_credential method=azure_cli


{<ReferenceFrame.CameraRightForward: 'right-forward'>, <ReferenceFrame.CameraFrontForward: 'front-forward'>, <ReferenceFrame.CameraLeftForward: 'left-forward'>}


INFO     2024-07-30 17:41:26,703 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=intrinsics Intrinsics(scale=tensor([745.7510, 744.3447]), shift=tensor([954.4684, 527.4114]))
INFO     2024-07-30 17:41:26,704 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=Image size: (1080, 1920, 3)
INFO     2024-07-30 17:41:26,808 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=camera_pose: [[ 0.94139916  0.3355943   0.03382378]
 [-0.33713743  0.93927765  0.06399902]
 [-0.01029221 -0.07165188  0.9973766 ]]
/home/benjin/.cache/bazel/_bazel_benjin/253920fd9b6cbb686fc69e16c3d2789e/execroot/WayveCode/bazel-out/k8-opt/bin/tools/jupyter_calibration.runfiles/pip-core_torch/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /build/torch/aten/src/ATen/native/T

Transform3D(rotation=RotationVector3D(forward_left_up=tensor([-0.0692,  0.0225, -0.3434]), frame=[Frameless]), translation=Vector3D(forward_left_up=tensor([31.8440, -0.6187,  1.5627]), frame=[Frameless]), from_frame=[Frameless], to_frame=[Frameless])


INFO     2024-07-30 17:41:27,092 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=camera_pose: [[ 0.94139916  0.3355943   0.03382378]
 [-0.33713743  0.93927765  0.06399902]
 [-0.01029221 -0.07165188  0.9973766 ]]
INFO     2024-07-30 17:41:27,256 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=intrinsics Intrinsics(scale=tensor([745.7510, 744.3447]), shift=tensor([954.4684, 527.4114]))
INFO     2024-07-30 17:41:27,256 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=Image size: (1080, 1920, 3)


...

Transform3D(rotation=RotationVector3D(forward_left_up=tensor([-0.0076,  0.0453,  0.3328]), frame=[Frameless]), translation=Vector3D(forward_left_up=tensor([31.8609,  0.6203,  1.5009]), frame=[Frameless]), from_frame=[Frameless], to_frame=[Frameless])


INFO     2024-07-30 17:41:30,170 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=camera_pose: [[ 9.4412369e-01 -3.2675007e-01  4.3183398e-02]
 [ 3.2640883e-01  9.4511080e-01  1.4929234e-02]
 [-4.5691222e-02  3.9803422e-07  9.9895561e-01]]
INFO     2024-07-30 17:41:30,331 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=intrinsics Intrinsics(scale=tensor([739.1251, 741.5219]), shift=tensor([958.0194, 529.6061]))
INFO     2024-07-30 17:41:30,332 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=Image size: (1080, 1920, 3)


Transform3D(rotation=RotationVector3D(forward_left_up=tensor([-0.0076,  0.0453,  0.3328]), frame=[Frameless]), translation=Vector3D(forward_left_up=tensor([31.8609,  0.6203,  1.5009]), frame=[Frameless]), from_frame=[Frameless], to_frame=[Frameless])


INFO     2024-07-30 17:41:30,432 wayve.services.calibration.tools.verify_calibration.manual_verification bev_protofile message=camera_pose: [[ 9.4412369e-01 -3.2675007e-01  4.3183398e-02]
 [ 3.2640883e-01  9.4511080e-01  1.4929234e-02]
 [-4.5691222e-02  3.9803422e-07  9.9895561e-01]]


Transform3D(rotation=RotationVector3D(forward_left_up=tensor([-0.0076,  0.0453,  0.3328]), frame=[Frameless]), translation=Vector3D(forward_left_up=tensor([31.8609,  0.6203,  1.5009]), frame=[Frameless]), from_frame=[Frameless], to_frame=[Frameless])
